# Analysis of spontaneous activity 
------
Neurons in the primary visual cortex may be active also in absence of visual stimulation. Spontaneous activity is crucial during cortical development, but is also present in adulthood.

#### Import packages

In [ ]:
from allensdk.core.brain_observatory_cache import BrainObservatoryCache
from allensdk.core.brain_observatory_nwb_data_set import BrainObservatoryNwbDataSet
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import math
import os
# import scipy
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from scipy.stats import zscore, spearmanr, pearsonr
from tqdm import tqdm


#### define helper functions

In [ ]:
def find_nearest(array,value):
    idx = np.searchsorted(array, value, side="left")
    if idx > 0 and (idx == len(array) or math.fabs(value - array[idx-1]) < math.fabs(value - array[idx])):
        return array[idx-1]
    else:
        return array[idx]

def nan_helper(y):
    """Helper to handle indices and logical indices of NaNs.

    Input:
        - y, 1d numpy array with possible NaNs
    Output:
        - nans, logical indices of NaNs
        - index, a function, with signature indices= index(logical_indices),
          to convert logical indices of NaNs to 'equivalent' indices
    Example:
        >>> # linear interpolation of NaNs
        >>> nans, x= nan_helper(y)
        >>> y[nans]= np.interp(x(nans), x(~nans), y[~nans])
    """

    return np.isnan(y), lambda z: z.nonzero()[0]

### Download the data

#### Neurophysiology data: Slc17a7-IRES2-Cre driver line
Exploring recordings from a mouse line expressing GCamP6 in excitatory neurons (recordings in VISp)

In [ ]:
filter_json = """
[
    {
        "field": "area",
        "op": "in",
        "value": [
            "VISp"
        ]
    },
    {
        "field": "tld1_name",
        "op": "in",
        "value": [
            "Slc17a7-IRES2-Cre"
        ]
    }
]
"""
       
filters = json.loads(filter_json)
boc = BrainObservatoryCache(manifest_file="brain_observatory/manifest.json")
cells = boc.get_cell_specimens(filters=filters)
cells_df = pd.DataFrame(cells)
cells_df

In [ ]:
cells_df.columns.unique().tolist()
cont_ids = cells_df['experiment_container_id'].unique()
experiments = boc.get_ophys_experiments(experiment_container_ids=cont_ids, include_failed=False, require_eye_tracking=True)
exp_ids = [d['id'] for d in experiments]
exp_ids = exp_ids[:20]

In [ ]:
saving_dir = r'D:\proj_Analysis-BrainObservatory\brain_observatory\ophys_experiment_data'
files = os.listdir(saving_dir)
exp_ids = [int(s[:-4]) for s in files]
exp_ids

In [ ]:
bad_ids = []
for exp in tqdm(exp_ids):
    try:
        boc.get_ophys_pupil_data(exp,  suppress_pupil_data=False)
        boc.get_ophys_experiment_data(exp)
    except Exception:
        bad_ids.append(exp)
        print('a')
# exp_ids = [id for id in exp_ids if id not in bad_ids]

for exp in tqdm(exp_ids):
    boc.get_ophys_experiment_data(exp)


In [ ]:
n =1


id = exp_ids[n]
exp  = boc.get_ophys_experiment_data(id)
dff = exp.get_dff_traces()
calcium = zscore(dff[1], axis = 1, ddof=1)
eye = boc.get_ophys_pupil_data(id, suppress_pupil_data=False)
stim = boc.get_ophys_experiment_stimuli(id)



In [ ]:
spont = exp.get_stimulus_table('spontaneous').values
spont

In [ ]:
id

In [ ]:
ts_blocks = []
for i in range(spont.shape[1]):
    ts_blocks.append(dff[0][spont[i,0]:spont[i,1]])
ts = np.hstack(ts_blocks)


In [ ]:
eye_ts_blocks =[]
eye_sp_blocks = []
for t in ts_blocks:
    eye_ts_blocks = [find_nearest(eye.index.tolist(), t[0]), find_nearest(eye.index.tolist(), t[-1])]
    eye_sp_blocks.append(eye.loc[eye_ts_blocks[0]:eye_ts_blocks[1]])
eye_sp = pd.concat(eye_sp_blocks, axis = 0)
eye_sp

In [ ]:
pupil = []
pupilraw = []
for eye_df, t in zip(eye_sp_blocks, ts_blocks):
    new_pupil = np.interp(t, eye_df.index.to_list(),  eye_df['filtered_pupil_area'])
    nans, x= nan_helper(new_pupil)
    new_pupil[nans]= np.interp(x(nans), x(~nans), new_pupil[~nans])
    new_pupil_1 = new_pupil.copy()
    new_pupil = pd.Series(new_pupil).rolling(60, min_periods=1).median().values

    new_pupil = zscore(new_pupil, ddof=1, nan_policy = 'omit')
    new_pupil_1 = zscore(new_pupil_1, ddof=1, nan_policy = 'omit')

    pupil.append(new_pupil)
    pupilraw.append(new_pupil_1)

pupil = np.hstack(pupil)
pupilraw = np.hstack(pupilraw)



In [ ]:
%matplotlib inline
plt.figure(figsize= (22,4))
plt.plot(pupilraw)
plt.plot(pupil)

# 
# plt.plot(eye_sp.index.tolist(), eye_sp['raw_pupil_area'])

In [ ]:
(spont[0,1]-spont[0,0])/3


In [ ]:
traces =[]
for i in range(spont.shape[0]):
    traces.append(calcium[:,spont[i,0] : spont[i,-1]])
traces = np.hstack(traces)


In [ ]:
pca = PCA()

components = pca.fit_transform(traces.T)
scree1 = np.cumsum(pca.explained_variance_ratio_)

pca.explained_variance_ratio_



In [ ]:


sns.set_context('talk')
f, ax = plt.subplots()

lastv = np.where((scree1<0.76) & (scree1>0.75))[0][0]
plt.axvline(x = lastv,ymin = 0, ymax=scree1[lastv], color = 'k', linewidth = 1, linestyle = '--', alpha = .7)
plt.axhline( y= .75,xmin =0 ,  xmax =lastv/traces.shape[0],color = 'k', linewidth = 1, linestyle = '--', alpha = .7)

sns.lineplot(y= scree1,x =  np.arange(start =1,stop =  traces.shape[0]+1))


ax.set_title('Dimensionality of spontaneous activity', y = 1.05)
ax.set_ylabel('Cumulative fraction\nof variance explained')
ax.set_xlabel('Number of components')

ax.set_xlim([1,traces.shape[0]+1])
ax.set_ylim([0,1])

plt.savefig('PCA_allen.svg', bbox_inches = 'tight', transparent = False)

In [ ]:
lastv

In [ ]:
bins = np.linspace(0,  18292, 31)
idx = [[int(a),int(b)] for a, b in zip(bins[0:-1], bins[1:]) if spont[0][1] not in np.arange(a,b)]
for i in idx:
    print(i)

In [ ]:
len(idx)


In [ ]:
plt.hist(digitized[0,:])

In [ ]:
f = plt.figure(figsize =(7,4), dpi = 600)
traces.shape
digitized = np.digitize(traces, bins)
plt.imshow(digitized)
# bin_means = [traces[:,digitized == i].mean() for i in range(1, len(bins))]
# bin_means

In [ ]:
dff_0 = traces[:,:-500].T
dff_1 = traces[:,-501:].T
eye_0 = pupil[:-500]
eye_1 = pupil[-501:]
train_dff, test_dff, train_pupil, test_pupil = train_test_split(dff_0, eye_0, random_state=42)

In [ ]:
# dff_0 = traces[:,:-500].T
# dff_1 = traces[:,-501:].T
# eye_0 = pupil[:-500]
# eye_1 = pupil[-501:]
# poly = PolynomialFeatures(degree=3, include_bias=False)
# poly_features = poly.fit_transform(dff_0)
# train_dff, test_dff, train_pupil, test_pupil = train_test_split(poly_features, eye_0, random_state=42)

In [ ]:
lr = LinearRegression()

lr.fit(train_dff, train_pupil)

In [ ]:
pupil_pred = lr.predict(dff_1)

mean_squared_error(eye_0,pupil_pred)

In [ ]:
r2_score(eye_1,pupil_pred)

In [ ]:
plt.subplots(figsize = (12,6))
plt.plot(eye_0)
plt.plot(pupil_pred, alpha = .4)

In [ ]:
r = Ridge(alpha=10, random_state = 42)

r.fit(train_dff, train_pupil)
pupil_pred = r.predict(dff_1)
print(r.score(test_dff, test_pupil))
plt.plot(eye_1)
plt.plot(pupil_pred)

In [ ]:
l = Lasso(random_state = 42)

l.fit(train_dff, train_pupil)
pupil_pred = l.predict(dff_1)
l.score(test_dff, test_pupil)

In [ ]:
plt.plot(eye_1)
plt.plot(pupil_pred)

In [ ]:
rf = RandomForestRegressor(n_estimators =300, random_state = 42, oob_score = True, n_jobs  =4, verbose = True)

rf.fit(train_dff, train_pupil)
pupil_pred = rf.predict(dff_1)
print(rf.score(test_dff, test_pupil))

plt.subplots(figsize =  (12, 4))
plt.plot(eye_1 ,linewidth = 1, alpha = .99)
plt.plot(pupil_pred, linewidth = 1, alpha = .4)
plt.plot(pd.Series(pupil_pred).rolling(7).median() ,linewidth = 1, alpha = .6, color = 'r')

In [ ]:
pupil_pred = rf.predict(dff_0)
print(rf.score(test_dff, test_pupil))

plt.subplots(figsize =  (12, 4))
plt.plot(eye_0 ,linewidth = 1, alpha = .99)
plt.plot(pupil_pred, linewidth = 1, alpha = .4)
plt.plot(pd.Series(pupil_pred).rolling(7).median() ,linewidth = 1, alpha = .6, color = 'r')

In [ ]:
r, p = spearmanr(components[:,0], new_pupil)


r**2

In [ ]:
p>0.05

In [ ]:
eye_xpos = np.interp(ts, eye_sp.index.to_list(),  eye_sp['filtered_screen_coordinates_spherical_x_deg'])
nans, x= nan_helper(eye_xpos)
eye_xpos[nans]= np.interp(x(nans), x(~nans), eye_xpos[~nans])
eye_xpos_1 = eye_xpos.copy()
eye_xpos = pd.Series(eye_xpos).rolling(30, min_periods=1).median().values

eye_xpos = zscore(eye_xpos, ddof=1, nan_policy = 'omit')
eye_xpos_1 = zscore(eye_xpos_1, ddof=1, nan_policy = 'omit')

In [ ]:
%matplotlib inline
plt.figure(figsize= (12,4))
plt.plot(ts, eye_xpos_1)
plt.plot(ts, eye_xpos)

In [ ]:
eyex_0 = eye_xpos_1[:-300]
eyex_1 = eye_xpos_1[-300:]
train_dff, test_dff, train_x, test_x = train_test_split(dff_0, eyex_0, random_state=42)

In [ ]:
r = Ridge(alpha=1, random_state = 42)

r.fit(train_dff, train_x)
x_pred = r.predict(dff_1)
print(r.score(test_dff, test_x))
plt.plot(eyex_1)
plt.plot(x_pred)

In [ ]:
rf = RandomForestRegressor(n_estimators =400, random_state = 42, oob_score = True, n_jobs  =12, verbose = True)

rf.fit(train_dff, train_x)
x_pred = rf.predict(dff_1)
rf.score(test_dff, test_x)

plt.subplots(figsize =  (12, 4))
plt.plot(eyex_1 ,linewidth = 1, alpha = .99)
plt.plot(x_pred, linewidth = 1, alpha = .4)
plt.plot(pd.Series(x_pred).rolling(7).median() ,linewidth = 1, alpha = .6, color = 'r')

In [ ]:
speed = exp.get_running_speed()


In [ ]:
abs_speed = zscore(np.abs(speed[0][spont[0]:spont[1]]), ddof = 1)
plt.plot(abs_speed)
r, p = spearmanr(components[:,0], abs_speed)


In [ ]:
speed = speed[spont[0]:spont]

In [ ]:
run_0 = abs_speed[:-300]
run_1 = abs_speed[-300:]
train_dff, test_dff, train_run, test_run = train_test_split(dff_0, run_0, random_state=42)

In [ ]:
r = Ridge(alpha=15, random_state = 42)

r.fit(train_dff, train_run)
run_pred = r.predict(dff_1)
print(r.score(test_dff, test_run))
plt.plot(run_1)
plt.plot(run_pred)

In [ ]:
rf = RandomForestRegressor(n_estimators =400, random_state = 42, oob_score = True, n_jobs  =12, verbose = True)

rf.fit(train_dff, train_run)
run_pred = rf.predict(dff_1)
rf.score(test_dff, test_run)

plt.subplots(figsize =  (12, 4))
plt.plot(run_1 ,linewidth = 1, alpha = .99)
plt.plot(run_pred, linewidth = 1, alpha = .4)
plt.plot(pd.Series(run_pred).rolling(7).median() ,linewidth = 1, alpha = .6, color = 'r')

In [ ]:
id

In [ ]:
dff[1].shape

In [ ]:
events = boc.get_ophys_experiment_events(id)
events.shape

In [ ]:
exp.get_stimulus_epoch_table()

In [ ]:
events_sp =[]
for i in range(spont.shape[1]):
    events_sp.append(events[:,spont[i,0] : spont[i,-1]])
events_sp = np.hstack(events_sp)


In [ ]:
import matplotlib.cm as cm

cmap = cm.get_cmap('twilight')
freq = ((events_sp>0).sum(axis = 1))/((ts_blocks[0][-1]-ts_blocks[0][0])+(ts_blocks[1][-1]-ts_blocks[1][0]))
sns.histplot(freq, color=cmap(0.6), alpha = .8)

plt.title('Frequency of spontaneous events')
plt.xlabel('Frequency (events/s)')

In [ ]:

amp = events_sp.sum(axis = 1) /np.count_nonzero(events_sp, axis =1)

sns.histplot(amp, color=cmap(0.3), alpha = .8)

plt.title('Amplitude of spontaneous events')
plt.xlabel('Amplitude ($\Delta$F/F)')

In [ ]:
times = []
blocks_idx = np.diff(spont)
for i in range(events_sp.shape[0]):
    tot_bool = events_sp[i,:]>0
    bol1 = np.zeros(tot_bool.shape, dtype = bool)
    bol1[:blocks_idx[0][0]] = tot_bool[:blocks_idx[0][0]]
    bol2 = np.zeros(tot_bool.shape, dtype = bool)
    bol2[blocks_idx[0][0]+1:] = tot_bool[blocks_idx[0][0]+1:]
    times.append(ts[bol1])
    times.append(ts[bol2])
interv = [np.diff(t) for t in times]

interv1 = np.hstack(interv)
f, ax = plt.subplots(figsize=(7, 7))
# ax.set(yscale="log")
# for intv in interv:
#     plt.figure()
sns.histplot(interv1, color=cmap(0.2), alpha = .9)

plt.title('Inter-event interval')
plt.xlabel('time (ms)')


In [ ]:
bol2.sum()

In [ ]:
a = boc.get_ophys_experiment_data(647155122)
a.get_stimulus_epoch_table()
f = a.get_dff_traces()[1][:,908:19468]
e = boc.get_ophys_experiment_events(647155122)[:,908:19468]

In [ ]:
# e = boc.get_ophys_experiment_events(647155122)
e

In [ ]:
plt.figure(figsize=(16,3))

plt.plot(e[11,1000:1800]-0.1)
plt.plot(e[0,1000:1800]-0.1)
plt.plot(zscore(f[11,1000:1800]))

In [ ]:
b = boc.get_ophys_experiment_analysis(647155122, 'drifting_gratings')

In [ ]:

boc.get_ophys_experiment_stimuli(exp_ids[2])
exp2 = boc.get_ophys_experiment_data(exp_ids[2])
exp2.get_stimulus_epoch_table()